In [1]:
# 이 예제는 `2. RNN 실습 - 영화평 분류`

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import SimpleRNN, Dense, Input, Embedding
# model = Sequential()
# model.add(Input(shape=(80,))) # 입력하는 영화평의 길이를 80으로 제한, 길면 자르고, 짧으면 zero padding
# model.add(Embedding(input_dim=10000, output_dim=32))
# model.add(SimpleRNN(64))
# model.add(Dense(2, activation='softmax')) # model.add(Dense(1, activation='sigmoid'))
# model.summary()

# 이 모델의 test set accuracy는 77%

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 32)            320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6208      
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 326338 (1.24 MB)
Trainable params: 326338 (1.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [1]:
# Transformer를 이용한 영화평 분류

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model

inputs = layers.Input(shape=(80,))

input_embedding = layers.Embedding(input_dim=10000, output_dim=32)(inputs)
positions = tf.range(start=0, limit=80)
pos_encoding = layers.Embedding(input_dim=80, output_dim=32)(positions)
pos_enc_output = pos_encoding + input_embedding

attention_output = layers.MultiHeadAttention(num_heads=2, key_dim=32)(pos_enc_output, pos_enc_output)
x = layers.add([pos_enc_output, attention_output])
x = layers.BatchNormalization()(x)

ffnn = Sequential([layers.Dense(64, activation="relu"), 
                   layers.Dense(32, activation="relu")])(x)
x = layers.add([ffnn, x])
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)

x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(2, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 80, 32)               320000    ['input_1[0][0]']             
                                                                                                  
 tf.__operators__.add (TFOp  (None, 80, 32)               0         ['embedding[0][0]']           
 Lambda)                                                                                          
                                                                                                  
 multi_head_attention (Mult  (None, 80, 32)               8416      ['tf.__operators__.add[0][

In [2]:
# model.compile(loss='binary_crossentropy', optimizer='adam',
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [3]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

17464789/17464789 [==============================] - 2s 0us/step
(25000,) (25000,) (25000,) (25000,)


In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train, maxlen=80, truncating='post', padding='post')
X_test_pad = pad_sequences(X_test, maxlen=80, truncating='post', padding='post')

In [5]:
%%time
model.fit(X_train_pad, y_train, epochs=10, batch_size=200)

Epoch 1/10
125/125 [==============================] - 22s 159ms/step - loss: 0.4931 - accuracy: 0.7553
Epoch 2/10
125/125 [==============================] - 20s 157ms/step - loss: 0.3187 - accuracy: 0.8650
Epoch 3/10
125/125 [==============================] - 19s 155ms/step - loss: 0.2596 - accuracy: 0.8918
Epoch 4/10
125/125 [==============================] - 20s 156ms/step - loss: 0.2091 - accuracy: 0.9110
Epoch 5/10
125/125 [==============================] - 19s 155ms/step - loss: 0.1664 - accuracy: 0.9244
Epoch 6/10
125/125 [==============================] - 19s 152ms/step - loss: 0.1386 - accuracy: 0.9362
Epoch 7/10
125/125 [==============================] - 19s 153ms/step - loss: 0.1096 - accuracy: 0.9528
Epoch 8/10
125/125 [==============================] - 19s 156ms/step - loss: 0.0911 - accuracy: 0.9635
Epoch 9/10
125/125 [==============================] - 19s 155ms/step - loss: 0.0650 - accuracy: 0.9760
Epoch 10/10
125/125 [==============================] - 20s 158ms/step - l

In [6]:
model.evaluate(X_test_pad, y_test)

782/782 [==============================] - 11s 13ms/step - loss: 1.3055 - accuracy: 0.6165


[1.3055187463760376, 0.6164799928665161]

In [7]:
print(X_test_pad.shape)

(25000, 80)


In [8]:
import numpy as np
pred = model.predict(X_test_pad)
# pred = (pred > 0.5).astype(int)
pred = np.argmax(pred, axis=1)

782/782 [==============================] - 10s 12ms/step


In [9]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, pred))

[[8570 3930]
 [5658 6842]]


In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.61648

In [11]:
# 옵티마이저를 sgd로 바꿔보세요. accuracy: 0.57784
# 전체 단어의 개수를 1000개로 바꿔보세요. accuracy:
# 영화평의 길이를 200개로 바꿔보세요. accuracy:
# pad_sequence의 truncating과 padding을 pre로 바꿔보세요. accuracy:

In [12]:
# 아래 택스트를 긍/부정 분류하세요.
text = "My God the actors who potrayed the VIP people cannot act. I cringed everytime they said a line. It felt like they were just reading them. Even the intonation was off. It was like when we were kids and had to read a play in class and we exagerated the intonation. Terrible, just awful."

In [13]:
word_index = imdb.get_word_index()
# print(word_index)
# {'fawn': 34701, 'tsukino': 52006, 'nunnery': 52007,

1641221/1641221 [==============================] - 0s 0us/step


In [14]:
word_to_idx = {k:(v+3) for k,v in word_index.items()}
word_to_idx["<PAD>"] = 0
word_to_idx["<START>"] = 1
word_to_idx["<UNK>"] = 2  # unknown
word_to_idx["<UNUSED>"] = 3

In [15]:
input_text = text.lower().split()
print(input_text)

['my', 'god', 'the', 'actors', 'who', 'potrayed', 'the', 'vip', 'people', 'cannot', 'act.', 'i', 'cringed', 'everytime', 'they', 'said', 'a', 'line.', 'it', 'felt', 'like', 'they', 'were', 'just', 'reading', 'them.', 'even', 'the', 'intonation', 'was', 'off.', 'it', 'was', 'like', 'when', 'we', 'were', 'kids', 'and', 'had', 'to', 'read', 'a', 'play', 'in', 'class', 'and', 'we', 'exagerated', 'the', 'intonation.', 'terrible,', 'just', 'awful.']


In [16]:
# 단어를 숫자로 변환합니다. 
# word_to_idx에 없는 단어는 2(<UNK>)로 지정하며, 
# 인덱스가 10000보다 크면 3(<UNUSED>)로 지정합니다.
def encoding(review_text):
  encoded = []
  for word in review_text:
    try:
      idx = word_to_idx[word]
      if idx>10000:
        encoded.append(3)
      else:
        encoded.append(idx)
    except:
      encoded.append(2)
  return encoded

input_encoded = encoding(input_text)

In [17]:
np.array(input_encoded)[np.newaxis, :]

array([[ 61, 558,   4, 156,  37,   2,   4,   3,  84, 566,   2,  13,   3,
          3,  36, 301,   6,   2,  12, 421,  40,  36,  71,  43, 886,   2,
         60,   4,   3,  16,   2,  12,  16,  40,  54,  75,  71, 362,   5,
         69,   8, 332,   6, 297,  11, 707,   5,  75,   3,   4,   2,   2,
         43,   2]])

In [18]:
input_pad = pad_sequences(
    np.array(input_encoded)[np.newaxis, :],
    maxlen=80, truncating='post', padding='post')

In [19]:
model.predict(input_pad) # LSTM을 이용한 인공신경망 모형은 부정(0)으로 분류함

1/1 [==============================] - 0s 20ms/step


array([[0.08314691, 0.9168531 ]], dtype=float32)